# 打开机器人摄像头

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event
from IPython.display import display
import ipywidgets.widgets as widgets
from nxbot import Robot,event,bgr8_to_jpeg
import traitlets
from traitlets.config.configurable import Configurable
import ipywidgets
import numpy as np
import cv2

## 2.创建摄像头RGB数据回调函数

In [ ]:
# 创建显示窗口
image_widget = widgets.Image(format='jpeg')
# 创建接收图像数据的函数
def on_new_image(evt):
    # 获取图像信息
    img_data = evt.dict['data']
    # 将图像大小变为（320，240）
    img_data = cv2.resize(img_data, (320,240))
    # 将图像信息传给显示窗口
    image_widget.value=bgr8_to_jpeg(img_data)

## 3.创建摄像头深度数据回调函数
摄像头深度信息由[640，480]的深度值矩阵构成。
1. 我们可以获取某一个点的深度值，通过滑块将这个点的深度值展示出来；
2. 我们也可以获取所有深度值，将深度值转为图像信息，通过图像的方式展示出来。

In [ ]:

# 创建滑块，将深度信息进行可视化
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='中心点深度值')

# 创建深度图像显示窗口
depth_image_widget = widgets.Image(format='jpeg')

def on_new_depth(evt):
    
    # 获取指定像素的深度值
    depth = evt.dict['data'].get_distance(320, 240)
    # 如果320，240这个点的深度值为0，我们就找它相邻的点
    if depth ==0:
        depth = evt.dict['data'].get_distance(322, 241)
        
    # 将深度值传给滑块
    depth_slider.value = depth
    # 获取摄像头所有的深度信息
    depth_frame = np.asanyarray(evt.dict['data'].get_data())
    # 将深度信息值转换为深度图
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_frame, alpha=0.03), cv2.COLORMAP_JET)
    # 改变深度图的大小为（320，240）
    show_img = cv2.resize(depth_colormap, (320,240))
    depth_image_widget.value = bgr8_to_jpeg(show_img)

## 4.连接小车，显示摄像头数据

In [ ]:
rbt = Robot()
rbt.connect()

# 腊肠狗可以打开深度摄像头
if rbt.name=='dachbot':
    # 打开摄像头,"enable_depth_stream=True" 表示打开摄像头深度信息
    rbt.camera.start(enable_depth_stream=True)
    # 监听摄像头深度数据，并将数据传给“on_new_depth”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_DEPTH,on_new_depth)
    # 监听摄像头图像数据，并将数据传给“on_new_image”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
    # 通过"display"模块将深度值显示在滑块上。
    image_box = ipywidgets.HBox([image_widget,depth_image_widget])
    # 显示窗口
    display(image_box)
    
# 桌面小车只有rbg图像信息    
elif rbt.name=='dbot':
    rbt.camera.start()
    # 监听摄像头图像数据，并将数据传给“on_new_image”函数
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
    # 显示窗口
    display(image_widget)

## 5.断开连接

In [ ]:
# rbt.disconnect()